### **README**

# What this does

This script allows you to download any number of **Looks** into a singular Excel file. Each Look will be a tab/worksheet within that Excel file. Each Looks title will be the title of the tab. You can also specify a file name for the download, instead of the default `look_name date_time.xlsx`. 

# Notes
`download_multilook_tabbed(input_mode,look_list, file_name)` accepts 3 inputs:

- `input_mode`(string) - accepts a value of `coded` or `otf`. 
  - `otf` prompts users for CLI input for a comma delimited list of look IDs. IDs should be in the format `'1','2','3',etc`
  - `coded` passes input from `look_list` into the download function. 
- `look_list`(list) - accepts a comma delimited list of Look IDs. IDs should be in the format `'1','2','3',etc`. Only used with `input_mode` of `coded`
- `file_name`(string) - the desired output file name of the final excel file.

If using colab version, you will need to input your api url, client_id, and client_secret in the `## colab inputs ##` section. 

```
os.environ['LOOKERSDK_BASE_URL'] = 'https://example.looker.com'
os.environ['LOOKERSDK_API_VERSION'] = '3.1'
os.environ['LOOKERSDK_CLIENT_ID'] = 'client id here'
os.environ['LOOKERSDK_CLIENT_SECRET'] = 'client secret here'
```

If using python (local) version, these values should be set for the sdk per either an `.ini` file or as an environment variable. See Python SDK documentation for more information. https://pypi.org/project/looker-sdk/

In [17]:
### inputs ###
# input_mode - coded or otf 
input_mode = 'coded | otf'
# look_list - if coded, passes through to list_create
look_list = []
# file_name - output file name of the xlsx file
file_name = ''

In [18]:
### colab specific ###
# install the sdk
!pip install looker-sdk
# set environ variables
import os

## colab inputs ##
os.environ['LOOKERSDK_BASE_URL'] = ''
os.environ['LOOKERSDK_API_VERSION'] = '3.1'
os.environ['LOOKERSDK_CLIENT_ID'] = ''
os.environ['LOOKERSDK_CLIENT_SECRET'] = ''

# will need these for downloading the file from colab
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials
# auth stuff for download
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [19]:
### import packages ###
# import and init
import looker_sdk
sdk = looker_sdk.init31("looker.ini")

import pandas as pd
import csv

def download_multilook_tabbed(input_mode,look_list, file_name):
  # local variables #
  # looks - returned by list_create and passed through to other functions
  looks = []
  # listlen - passed through to downloader and title_get to set number of iterations
  listlen = 0
  # titles - returned by title_get and passed through into downloader
  titles = []
  # helper methods
  def list_create(input_mode, look_list):
    if input_mode == 'coded':
      looks.extend(look_list)
      return looks
    elif input_mode == 'otf':
      inp = input('comma delimited list of looks please').split(',')
      looks.extend(inp)
      return looks
    else:
      print('please select an input_mode')

  def title_get(looks, lislen, titles):
    for i in range(lislen):
      data = sdk.look(looks[i])
      title = data.title
      titles.append(title)
    return titles

  def downloader(looks, lislen, titles, file_name):
    with pd.ExcelWriter(file_name+'.xlsx') as writer: 
      for i in range(lislen):
        data = sdk.run_look(looks[i],'csv')
        with open('temp.csv', 'w', newline='') as file:
            file.write(data)
        df = pd.read_table('temp.csv', delimiter =",",engine='python')
        df.to_excel(writer, sheet_name=(titles[i]))

  # wiring it up
  list_create(input_mode,look_list)
  lislen = len(looks)
  title_get(looks, lislen, titles)
  downloader(looks, lislen, titles, file_name)

download_multilook_tabbed(input_mode,look_list, file_name)

In [20]:
# colab specific - download the file
from google.colab import files 
files.download(file_name+'.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>